# Test Atlas v2

See atlas v1 issues:
https://github.com/roocs/clisops/issues/317

In [1]:
import xarray as xr
from clisops.core import subset
from clisops.ops import subset as subset_op
from clisops.utils.dataset_utils import open_xr_dataset

import time
import os

In [2]:
basedir_atlas_v2 = "/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset"

cmip6_nc = f"{basedir_atlas_v2}/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc"
cmip6_nc

'/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc'

## ncdump - CMIP6

In [3]:
! du -sh {cmip6_nc}

5.6G	/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc


In [4]:
! ncdump -sh {cmip6_nc}

netcdf t_CMIP6_historical_mon_185001-201412_v02 {
dimensions:
	bnds = 2 ;
	lon = 360 ;
	lat = 180 ;
	time = 1980 ;
	member = 30 ;
variables:
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "latitude" ;
		lat:bounds = "lat_bnds" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_Fletcher32 = "true" ;
		lat:_Shuffle = "true" ;
		lat:_DeflateLevel = 1 ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_Fletcher32 = "true" ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_DeflateLevel = 1 ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "longitude" ;
		lon:bounds = "lon_bnds" ;
		lon:_Storage = "chunked" ;
		lon:_ChunkSizes = 360 ;
		lon:_Fletcher32 = "true" ;
		lon:_Shuffle = "true" ;
		lon:_DeflateLevel = 1

## helper functions

In [5]:
def ds_info(ds):
    # compression levels
    print("data vars with compression:")
    for var in ds.data_vars:
        # print(var, ds[var].encoding, "\n")
        complevel = ds[var].encoding.get("complevel", 0)
        print(var, "compression level =", complevel)
        
    # fill values
    print("\nfill values:")
    var_list = list(ds.coords) + list(ds.data_vars)
    for var in var_list:
        fill_value = ds[var].encoding.get("_FillValue")
        print(var, "fill value =", fill_value)
        
    # string attributes with compression
    print("\nstring attributes with compression:")
    for cvar in [
            "member_id",
            "gcm_variant",
            "gcm_model",
            "gcm_institution",
            "rcm_variant",
            "rcm_model",
            "rcm_institution",
        ]:
            for en in ["zlib", "shuffle", "complevel"]:
                try:
                    print(cvar, en, ds[cvar].encoding[en])
                except KeyError:
                    pass

## xarray - CMIP6

TODO: still using two fill values for variable t. Without a fix it is not possible to write as netcdf file.

Error Message: 
Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.

In [6]:
ds_cmip6 = xr.open_dataset(cmip6_nc)
ds_cmip6

/home/k/k204199/.envs/clisops/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 't' has multiple fill values {np.float32(1.0384594e+34), np.float32(-1.7014118e+38)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.Dataset> Size: 15GB
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 1kB -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time             (time) datetime64[ns] 16kB 1850-01-01 ... 2014-12-01
    member_id        (member) <U46 6kB ...
    gcm_institution  (member) <U19 2kB ...
    gcm_model        (member) <U17 2kB ...
    gcm_variant      (member) <U8 960B ...
    height2m         float64 8B ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 3kB ...
    lon_bnds         (lon, bnds) float64 6kB ...
    time_bnds        (time, bnds) datetime64[ns] 32kB ...
    t                (member, time, lat, lon) float32 15GB ...
    crs              int32 4B ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [7]:
ds_info(ds_cmip6)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = -1.7014118e+38
crs fill value = None

string attributes with compression:


In [8]:
ds = ds_cmip6.isel(time=slice(0, 12), lon=slice(30, 50), lat=slice(50, 70))
ds

<xarray.Dataset> Size: 588kB
Dimensions:          (lat: 20, bnds: 2, lon: 20, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 160B -39.5 -38.5 -37.5 ... -22.5 -21.5 -20.5
  * lon              (lon) float64 160B -149.5 -148.5 -147.5 ... -131.5 -130.5
  * time             (time) datetime64[ns] 96B 1850-01-01 ... 1850-12-01
    member_id        (member) <U46 6kB ...
    gcm_institution  (member) <U19 2kB ...
    gcm_model        (member) <U17 2kB ...
    gcm_variant      (member) <U8 960B ...
    height2m         float64 8B ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 320B ...
    lon_bnds         (lon, bnds) float64 320B ...
    time_bnds        (time, bnds) datetime64[ns] 192B ...
    t                (member, time, lat, lon) float32 576kB ...
    crs              int32 4B ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [9]:
# clean up outputs

! rm /tmp/output_*

In [10]:
try:
    ds.to_netcdf("/tmp/output_atlas_v2_cmip6_xarray.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)
else:
    print("it worked!")

Fails to write as netcdf! Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.


In [11]:
# fix for atlas
# see: https://github.com/roocs/clisops/issues/317

def fix_atlas(ds):
    var_list = list(ds.coords) + list(ds.data_vars)
    # Remove fill-values
    for var in var_list:
        ds[var].encoding["_FillValue"] = None
    # Remove string deflation options if applicable
    for cvar in [
        "member_id",
        "gcm_variant",
        "gcm_model",
        "gcm_institution",
        "rcm_variant",
        "rcm_model",
        "rcm_institution",
    ]:
        for en in ["zlib", "shuffle", "complevel"]:
            try:
                del ds[cvar].encoding[en]
            except KeyError:
                pass
    return ds

In [12]:
# need to apply fixes to apply operations and save to netcdf

start = time.time()

ds = fix_atlas(ds)
ds.to_netcdf("/tmp/atlas_v2_cmip6_4.nc")

duration = time.time() - start
print(f"duration: {duration} secs")

duration: 0.4513092041015625 secs


## clisops-ops - cmip6 - subset

TODO: subset bbox is not possible! It uses a lot of memory ... even my 32GB VM was not enough for a successful run.

We need to patch `clisops.utils.dataset_utils.is_time()` ... the function `np.atleast1d` leads to this memory overflow.


In [13]:
# show clisops version
import clisops

print(clisops.__version__)

0.15.0


In [14]:
# monkey patch for clisops is_time()

from clisops.utils import dataset_utils

def custom_is_time(coord):
    print(f"Custom behavior for is_time with input: {coord.name}")
    
    import numpy as np
    
    if "time" in coord.cf.coordinates and coord.name in coord.cf.coordinates["time"]:
        return True

    if (
        "time" in coord.cf.standard_names
        and coord.name in coord.cf.standard_names["time"]
    ):
        return True

    if np.issubdtype(coord.dtype, np.datetime64):
        return True

    # TODO: this code leads to memory overflow when applied on a data variable!
    print(f"skip np.atleast_1d(coord.values) on: {coord.name}")
    _check_coord = False
    if _check_coord and isinstance(np.atleast_1d(coord.values)[0], cftime.datetime):
        return True

    if hasattr(coord, "axis"):
        if coord.axis == "T":
            return True

    return False


# Monkey patch the function
dataset_utils.is_time = custom_is_time

In [15]:
ds_cmip6 = open_xr_dataset(cmip6_nc)

/home/k/k204199/.envs/clisops/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 't' has multiple fill values {np.float32(1.0384594e+34), np.float32(-1.7014118e+38)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [16]:
# clean up outputs

! rm /tmp/output_*

In [17]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2012-01-01/2012-12-31",
    # time_components="year:2013",
    area=(0.0, 50.0, 10.0, 60.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

Custom behavior for is_time with input: time
Custom behavior for is_time with input: member_id
skip np.atleast_1d(coord.values) on: member_id
Custom behavior for is_time with input: gcm_institution
skip np.atleast_1d(coord.values) on: gcm_institution
Custom behavior for is_time with input: gcm_model
skip np.atleast_1d(coord.values) on: gcm_model
Custom behavior for is_time with input: gcm_variant
skip np.atleast_1d(coord.values) on: gcm_variant
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: time_bnds
skip np.atleast_1d(coord.values) on: time_bnds
Custom behavior for is_time with input: t
skip np.atleast_1d(coord.values) on: t
Custom behavior for is_time with input: crs
skip np.atleast_1d(coord.values) on: crs
Custom behavior for is_time with input: time
Custom behavior for is_time with input: member_id
ski

'/tmp/output_001.nc'

In [18]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 0.1149148941040039 MB


In [19]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset> Size: 156kB
Dimensions:          (lat: 10, bnds: 2, lon: 10, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 80B 50.5 51.5 52.5 53.5 ... 57.5 58.5 59.5
  * lon              (lon) float64 80B 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * time             (time) datetime64[ns] 96B 2012-01-01 ... 2012-12-01
    member_id        (member) <U46 6kB ...
    gcm_institution  (member) <U19 2kB ...
    gcm_model        (member) <U17 2kB ...
    gcm_variant      (member) <U8 960B ...
    height2m         float64 8B ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 160B ...
    lon_bnds         (lon, bnds) float64 160B ...
    time_bnds        (time, bnds) datetime64[ns] 192B ...
    t                (member, time, lat, lon) float32 144kB ...
    crs              int32 4B ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

## clisops-ops - ERA5 - subset

In [20]:
era5_nc = f"{basedir_atlas_v2}/ERA5/t_ERA5_mon_194001-202312_v02.nc"
era5_nc

'/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/ERA5/t_ERA5_mon_194001-202312_v02.nc'

In [21]:
ds_era5 = open_xr_dataset(era5_nc)
ds_era5

/home/k/k204199/.envs/clisops/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 't' has multiple fill values {np.float32(1.0384594e+34), np.float32(-1.7014118e+38)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.Dataset> Size: 4GB
Dimensions:    (lat: 721, bnds: 2, lon: 1440, time: 1008)
Coordinates:
  * lat        (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon        (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time       (time) object 8kB 1940-01-01 00:00:00 ... 2023-12-01 00:00:00
    height2m   float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lat, bnds) float64 12kB ...
    lon_bnds   (lon, bnds) float64 23kB ...
    time_bnds  (time, bnds) object 16kB ...
    t          (time, lat, lon) float32 4GB ...
    crs        int32 4B ...
Attributes: (12/25)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.125
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-09 11:35:45.167894+01:00
    tracking_id:                67ddc63f-1801-478d-bba9-bdf8c6588dd7

In [22]:
# clean up outputs

! rm /tmp/output_*

In [23]:
start = time.time()

outputs = subset_op(
    ds=ds_era5,
    time="2012-01-01/2012-12-31",
    # time_components="year:2013",
    area=(0.0, 50.0, 10.0, 60.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

Custom behavior for is_time with input: time
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: time_bnds
skip np.atleast_1d(coord.values) on: time_bnds
Custom behavior for is_time with input: t
skip np.atleast_1d(coord.values) on: t
Custom behavior for is_time with input: crs
skip np.atleast_1d(coord.values) on: crs
Custom behavior for is_time with input: time
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: time_bnds
skip np.atleast_1d(coord.values) on: time_bnds
Custom behavior for is_time with input: t
skip np.atleast_1d(coord.values) on: t
Custom behavior for is_time with input: crs
skip np.atleast_1d(coord.values) on: crs
dura

'/tmp/output_001.nc'

In [24]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset> Size: 83kB
Dimensions:    (lat: 41, bnds: 2, lon: 41, time: 12)
Coordinates:
  * lat        (lat) float64 328B 50.0 50.25 50.5 50.75 ... 59.5 59.75 60.0
  * lon        (lon) float64 328B 0.0 0.25 0.5 0.75 1.0 ... 9.25 9.5 9.75 10.0
  * time       (time) datetime64[ns] 96B 2012-01-01 2012-02-01 ... 2012-12-01
    height2m   float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lat, bnds) float64 656B ...
    lon_bnds   (lon, bnds) float64 656B ...
    time_bnds  (time, bnds) datetime64[ns] 192B ...
    t          (time, lat, lon) float32 81kB ...
    crs        int32 4B ...
Attributes: (12/25)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.125
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-09 11:35:45.167894+01:00
    tracking_id:                67ddc63f-1801-478d-bba9-bdf8c6588dd7

## clisops-ops - CORDEX-EUR-11 - subset


In [25]:
cordex_eur11_nc = f"{basedir_atlas_v2}/CORDEX-EUR-11/historical/t_CORDEX-EUR-11_historical_mon_197001-200512_v02.nc"
cordex_eur11_nc

'/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CORDEX-EUR-11/historical/t_CORDEX-EUR-11_historical_mon_197001-200512_v02.nc'

In [26]:
ds_cordex_eur11 = open_xr_dataset(cordex_eur11_nc)
ds_cordex_eur11

/home/k/k204199/.envs/clisops/lib/python3.12/site-packages/xarray/conventions.py:193: SerializationWarning: variable 't' has multiple fill values {np.float32(1.0384594e+34), np.float32(-1.7014118e+38)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.Dataset> Size: 44GB
Dimensions:          (lat: 408, bnds: 2, lon: 881, time: 432, member: 71)
Coordinates:
  * lat              (lat) float64 3kB 21.81 21.94 22.06 ... 72.44 72.56 72.69
  * lon              (lon) float64 7kB -44.81 -44.69 -44.56 ... 65.06 65.19
  * time             (time) object 3kB 1970-01-01 00:00:00 ... 2005-12-01 00:...
    member_id        (member) <U60 17kB ...
    gcm_institution  (member) <U12 3kB ...
    gcm_model        (member) <U12 3kB ...
    gcm_variant      (member) <U7 2kB ...
    rcm_institution  (member) <U10 3kB ...
    rcm_model        (member) <U17 5kB ...
    rcm_variant      (member) <U3 852B ...
    height2m         float64 8B ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 7kB ...
    lon_bnds         (lon, bnds) float64 14kB ...
    time_bnds        (time, bnds) object 7kB ...
    t                (member, time, lat, lon) float32 44GB ...
    crs              int32 4B ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -44.875
    geospatial_lon_max:         65.25
    geospatial_lon_resolution:  0.125
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-09 16:29:21.663841+01:00
    tracking_id:                f9670cb4-5ffa-4c90-859b-b54bffae0ce4

In [27]:
# clean up outputs

! rm /tmp/output_*

In [28]:
start = time.time()

outputs = subset_op(
    ds=ds_cordex_eur11,
    time="2000-01-01/2000-12-31",
    # time_components="year:2013",
    area=(0.0, 50.0, 10.0, 60.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

Custom behavior for is_time with input: time
Custom behavior for is_time with input: member_id
skip np.atleast_1d(coord.values) on: member_id
Custom behavior for is_time with input: gcm_institution
skip np.atleast_1d(coord.values) on: gcm_institution
Custom behavior for is_time with input: gcm_model
skip np.atleast_1d(coord.values) on: gcm_model
Custom behavior for is_time with input: gcm_variant
skip np.atleast_1d(coord.values) on: gcm_variant
Custom behavior for is_time with input: rcm_institution
skip np.atleast_1d(coord.values) on: rcm_institution
Custom behavior for is_time with input: rcm_model
skip np.atleast_1d(coord.values) on: rcm_model
Custom behavior for is_time with input: rcm_variant
skip np.atleast_1d(coord.values) on: rcm_variant
Custom behavior for is_time with input: lat_bnds
skip np.atleast_1d(coord.values) on: lat_bnds
Custom behavior for is_time with input: lon_bnds
skip np.atleast_1d(coord.values) on: lon_bnds
Custom behavior for is_time with input: time_bnds
skip

'/tmp/output_001.nc'

In [29]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset> Size: 22MB
Dimensions:          (lat: 80, bnds: 2, lon: 80, time: 12, member: 71)
Coordinates:
  * lat              (lat) float64 640B 50.06 50.19 50.31 ... 59.69 59.81 59.94
  * lon              (lon) float64 640B 0.0625 0.1875 0.3125 ... 9.812 9.938
  * time             (time) datetime64[ns] 96B 2000-01-01 ... 2000-12-01
    member_id        (member) <U60 17kB ...
    gcm_institution  (member) <U12 3kB ...
    gcm_model        (member) <U12 3kB ...
    gcm_variant      (member) <U7 2kB ...
    rcm_institution  (member) <U10 3kB ...
    rcm_model        (member) <U17 5kB ...
    rcm_variant      (member) <U3 852B ...
    height2m         float64 8B ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 1kB ...
    lon_bnds         (lon, bnds) float64 1kB ...
    time_bnds        (time, bnds) datetime64[ns] 192B ...
    t                (member, time, lat, lon) float32 22MB ...
    crs              int32 4B ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -44.875
    geospatial_lon_max:         65.25
    geospatial_lon_resolution:  0.125
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-09 16:29:21.663841+01:00
    tracking_id:                f9670cb4-5ffa-4c90-859b-b54bffae0ce4